# Fusion using trectools package

The score for documents not present in a TrecRun is set to zero

In [1]:
from trectools import TrecRun, TrecEval, fusion

In [ ]:
print("hi")

In [2]:
r1 = TrecRun("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs-test/DPR-Reranked.run")

In [3]:
r2 = TrecRun("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs-test/Fusion_DPR_Reranked_w_Baseline_rrf_k_60.run")

In [4]:
r3 = TrecRun("/work/snaseri_umass_edu/better_P3/data/BETTER_PHASE3_COMBO_ARABIC_FARSI_RUSSIAN/REPRODUCE-GDPR-S1-AUTO/62e171cedebb2f0d2619f2f6.FINAL.out")

In [ ]:
r4 = TrecRun("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs/trectoolsFusion_hint_e2e_and_Baseline.run")

In [ ]:
r5 = TrecRun("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs/Fusion_DPR_Reranked_w_Baseline-better_system_output_rrf_k_60.run")

In [ ]:
r6 = TrecRun('/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs/Fusion_DPR_Reranked_w_Baseline_hint_e2e_rrf_k_60.run')

In [4]:
r7 = TrecRun('/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs/DPR-Reranked_baseline-concat_62e171cedebb2f0d2619f2f6.FINAL.out_hint_e2e.run')

In [5]:
fused_run = fusion.reciprocal_rank_fusion([r2, r3])

In [6]:
fused_run.print_subset("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs-test/Fusion_baseline_and_Fusion_DPR_Reranked_w_Baseline_rrf_k_60.run", topics=fused_run.topics())

File /work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs-test/Fusion_baseline_and_Fusion_DPR_Reranked_w_Baseline_rrf_k_60.run writen.


# Fusion using my own script

In [ ]:
from tqdm import tqdm

In [ ]:
from collections import defaultdict

In [ ]:
import os

In [ ]:
def read_run(path):
    run = {}
    with open(path) as f:
        for line in tqdm(f):
            qid, _ , did, _ , score ,_ = line.strip().split()
            score = float(score)
            run.setdefault(qid, {})
            run[qid][did] = score
    return run

In [ ]:
rerank_run = read_run("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs/zhiqi/hint_e2e.run")

baseline_run = read_run("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs/Fusion_DPR-Reranker_Baseline_rrf-k_60.run")
        
baseline_list = {}
for qid in baseline_run:
    ranklist = list(sorted(baseline_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
    baseline_list.setdefault(qid, [])
    for retdoc in ranklist:
        baseline_list[qid].append(retdoc[0])

In [ ]:
rrf_k = 60

In [ ]:
fusion_run = defaultdict(dict)
for qid in tqdm(rerank_run):
    rerank_list = list(sorted(rerank_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
    for i in range(len(rerank_list)):
        did = rerank_list[i][0]
        rerank_at = i + 1
        baseline_at = baseline_list[qid].index(did) + 1
        # print(rerank_at, baseline_at)
        fused_score = (1/(rrf_k + rerank_at)) + (1/(rrf_k + baseline_at))
        fusion_run[qid][did] = fused_score

In [ ]:
fusion_file = os.path.join("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs", f'fusion_ru_ar_reranker_dp_and_ru_ar_baseline_rrf_{rrf_k}.run')
with open(fusion_file, 'wt') as runfile:
    for qid in fusion_run:
        dummy_score = 100000
        fused_scores = list(sorted(fusion_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
        for i, (did, score) in enumerate(fused_scores):
            runfile.write(f'{qid} 0 {did} {i+1} {dummy_score} run\n')
            dummy_score -= 1